In [1]:
import os
import sys
import tensorflow                             as tf
import numpy                                  as np
import pandas                                 as pd
import matplotlib.pyplot                      as plt

In [2]:
WORKSPACE_PATH    = os.getenv('WORKSPACE_PATH')  
TVec              = np.array([1500.0, 5000.0, 7500.0, 10000.0, 12000.0, 15000.0, 20000.0])
NPerGroup         = 1

# TempVec = [1500.0, 5000.0, 10000.0, 15000.0, 20000.0]
# Molecule          = 'O2'
# PathToDiatPot     = WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/O2/UMN/FromUMN_Sorted.inp'
# # PathToTrainPts     = WORKSPACE_PATH + '/SurQCT/surqct/exec/residual_pts/'
# # PathToValidPts     = WORKSPACE_PATH + '/SurQCT/surqct/exec/residual_pts/'
# PathToTrainPts     = WORKSPACE_PATH + '//PRODE/run/SurQCT/o3inel/data/train/'
# PathToValidPts     = WORKSPACE_PATH + '//PRODE/run/SurQCT/o3inel/data/valid/'

TempVec = [1500.0, 5000.0, 7500.0, 10000.0, 12000.0, 15000.0, 20000.0]
Molecule          = 'N2'
PathToDiatPot     = WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp'
# PathToTrainPts     = WORKSPACE_PATH + '/SurQCT/surqct/exec/residual_pts/'
# PathToValidPts     = WORKSPACE_PATH + '/SurQCT/surqct/exec/residual_pts/'
PathToTrainPts     = WORKSPACE_PATH + '//PRODE/run/SurQCT/n3inel/data/train/'
PathToValidPts     = WORKSPACE_PATH + '//PRODE/run/SurQCT/n3inel/data/valid/'

# Molecule          = 'CO'
# PathToDiatPot     = WORKSPACE_PATH + '/CoarseAIR/coarseair/dtb/Molecules/CO/NASA/CO_levels_venturi.dat'
# PathTo3AtomsFldr  = WORKSPACE_PATH + '/Air_Database/Run_0D/database/levels/AmalInel_Sampled/'
# PathToGroups      = WORKSPACE_PATH + '/Air_Database/Run_0D/database/grouping/CO2_NASA/CO/LevelsMap_InelAmal83.csv'
kB = 8.617333262e-5


In [3]:
def compute_rotenergy(eint, vqn, jqn):
    NLevels = len(eint)
    EVib    = np.zeros((NLevels,1))
    ERot    = np.zeros((NLevels,1))
    ETemp   = np.zeros((np.amax(jqn)+1,1))
    for iLevel in range(NLevels):
        if (vqn[iLevel] == 0):
            ERot[iLevel]       = eint[iLevel]
            ETemp[jqn[iLevel]] = eint[iLevel]
        else:
            ERot[iLevel] = ETemp[jqn[iLevel]]
            EVib[iLevel] = eint[iLevel] - ERot[iLevel]
    return EVib, ERot

def read_diatdata(DataFile, Molecule, TTranVecTrain, TTranVecTest):
    print('[SurQCT]:   Reading Molecular Levels Data from: ' + DataFile)

    # Extracting all the rot-vib levels
    xMat           = pd.read_csv(DataFile, delim_whitespace=True, skiprows=15, header=None)
    xMat.columns   = ['vqn','jqn','EInt','egam','rMin','rMax','VMin','VMax','Tau','ri','ro']    

    # Shifting Energies so that Zero is the Min of Diatomic Potential at J=0
    xMat['EInt']   = ( xMat['EInt'].to_numpy() - np.amin(xMat['VMin'].to_numpy()) ) * 27.211399
    xMat['VMax']   = ( xMat['VMax'].to_numpy() - np.amin(xMat['VMin'].to_numpy()) ) * 27.211399
    xMat['VMin']   = ( xMat['VMin'].to_numpy() - np.amin(xMat['VMin'].to_numpy()) ) * 27.211399

    EVib, ERot     = compute_vibenergy(xMat['EInt'].to_numpy(), xMat['vqn'].to_numpy(int), xMat['jqn'].to_numpy(int))
    xMat['EVib']   = EVib
    xMat['ERot']   = ERot

    xMat['g']      = compute_degeneracy(xMat['jqn'].to_numpy(int), Molecule) 

    kB = 8.617333262e-5
    
    for TTran in TTranVecTrain:
        Str       = 'q_'+str(int(TTran))
        xMat[Str] = xMat['g'].to_numpy() * np.exp( - xMat['EInt'].to_numpy() / (kB*TTran) )

    for TTran in TTranVecTest:
        Str       = 'q_'+str(int(TTran))
        xMat[Str] = xMat['g'].to_numpy() * np.exp( - xMat['EInt'].to_numpy() / (kB*TTran) )

    return xMat


def compute_vibenergy(eint, vqn, jqn):
    NLevels = len(eint)
    EVib    = np.zeros((NLevels,1))
    ERot    = np.zeros((NLevels,1))
    ETemp   = np.zeros((np.amax(vqn)+1,1))
    for iLevel in range(NLevels):
        if (jqn[iLevel] == 0):
            EVib[iLevel]       = eint[iLevel]
            ETemp[vqn[iLevel]] = eint[iLevel]
        else:
            EVib[iLevel] = ETemp[vqn[iLevel]]
            ERot[iLevel] = eint[iLevel] - EVib[iLevel]
    return EVib, ERot


def compute_degeneracy(jqn, Molecule):
    if (Molecule == 'N2'):
        #g = (2*jqn + 1)
        NLevels = len(jqn)
        g       = np.zeros(NLevels)
        for iLevel in range(NLevels):
            if (jqn[iLevel] % 2 == 0):
                g[iLevel] = 1/2*(2*jqn[iLevel] + 1)
            else:
                g[iLevel] = 1/2*(2*jqn[iLevel] + 1)
    elif (Molecule == 'O2'):
        g = (2*jqn + 1)
    elif (Molecule == 'NO'):
        g = (2*jqn + 1)
    elif (Molecule == 'CO'):
        g = (2*jqn + 1)
    elif (Molecule == 'CN'):
        g = (2*jqn + 1)
    else:
        print('\n\nDEGENERACY NOT IMPLEMENTED FOR THIS MOLECULE!\n\n')

    return g

In [4]:
# Read Diatomic Potentials
DiatData   = []
NMolecules = 1

for iMol in range(NMolecules):
    DiatDataTemp = read_diatdata(PathToDiatPot, Molecule, TempVec, TempVec)
    DiatData.append(DiatDataTemp)
    NLevels = np.size(DiatData[iMol]['EInt'].to_numpy())


[SurQCT]:   Reading Molecular Levels Data from: /home/venturi/WORKSPACE//CoarseAIR/coarseair/dtb/Molecules/N2/LeRoy/MyLeroy_FromRobyn.inp


In [5]:
trainPts = pd.read_csv((PathToTrainPts + 'res.csv'), skiprows=1, header=None)
trainPts.columns   = ['log_EVib_i','log_ERot_i','ri_i','log_rorMin_i','TTran_i','log_EVib_Delta','log_ERot_Delta','ri_Delta','log_rorMin_Delta','TTran_Delta','KExcit','Idx_i','Idx_j','yWeights','qi','qj','partition_function_ratio']
validPts = pd.read_csv((PathToValidPts + 'res.csv'), skiprows=1, header=None)
validPts.columns   = ['log_EVib_i','log_ERot_i','ri_i','log_rorMin_i','TTran_i','log_EVib_Delta','log_ERot_Delta','ri_Delta','log_rorMin_Delta','TTran_Delta','KExcit','Idx_i','Idx_j','yWeights','qi','qj','partition_function_ratio']


In [7]:
# Mask = trainPts['partition_function_ratio']>1e-3
# trainPtsnew = pd.DataFrame()
# trainPtsnew['log_EVib_i'] = trainPts['log_EVib_i'].to_numpy()[Mask]
# trainPtsnew['log_ERot_i'] = trainPts['log_ERot_i'].to_numpy()[Mask]
# trainPtsnew['ri_i'] = trainPts['ri_i'].to_numpy()[Mask]
# trainPtsnew['log_rorMin_i'] = trainPts['log_rorMin_i'].to_numpy()[Mask]
# trainPtsnew['TTran_i'] = trainPts['TTran_i'].to_numpy()[Mask]
# trainPtsnew['log_EVib_Delta'] = trainPts['log_EVib_Delta'].to_numpy()[Mask]
# trainPtsnew['log_ERot_Delta'] = trainPts['log_ERot_Delta'].to_numpy()[Mask]
# trainPtsnew['ri_Delta'] = trainPts['ri_Delta'].to_numpy()[Mask]
# trainPtsnew['log_rorMin_Delta'] = trainPts['log_rorMin_Delta'].to_numpy()[Mask]
# trainPtsnew['TTran_Delta'] = trainPts['TTran_Delta'].to_numpy()[Mask]
# trainPtsnew['KExcit'] = trainPts['KExcit'].to_numpy()[Mask]
# trainPtsnew['Idx_i'] = trainPts['Idx_i'].to_numpy()[Mask]
# trainPtsnew['Idx_j'] = trainPts['Idx_j'].to_numpy()[Mask]
# trainPtsnew['yWeights'] = trainPts['yWeights'].to_numpy()[Mask]
# trainPtsnew['qi'] = trainPts['qi'].to_numpy()[Mask]
# trainPtsnew['qj'] = trainPts['qj'].to_numpy()[Mask]
# trainPtsnew['partition_function_ratio'] = trainPts['partition_function_ratio'].to_numpy()[Mask]
# trainPtsnew.to_csv(PathToTrainPts + '/res.csv', index=False)

# Mask = validPts['partition_function_ratio']>1e-3
# validPtsnew = pd.DataFrame()
# validPtsnew['log_EVib_i'] = validPts['log_EVib_i'].to_numpy()[Mask]
# validPtsnew['log_ERot_i'] = validPts['log_ERot_i'].to_numpy()[Mask]
# validPtsnew['ri_i'] = validPts['ri_i'].to_numpy()[Mask]
# validPtsnew['log_rorMin_i'] = validPts['log_rorMin_i'].to_numpy()[Mask]
# validPtsnew['TTran_i'] = validPts['TTran_i'].to_numpy()[Mask]
# validPtsnew['log_EVib_Delta'] = validPts['log_EVib_Delta'].to_numpy()[Mask]
# validPtsnew['log_ERot_Delta'] = validPts['log_ERot_Delta'].to_numpy()[Mask]
# validPtsnew['ri_Delta'] = validPts['ri_Delta'].to_numpy()[Mask]
# validPtsnew['log_rorMin_Delta'] = validPts['log_rorMin_Delta'].to_numpy()[Mask]
# validPtsnew['TTran_Delta'] = validPts['TTran_Delta'].to_numpy()[Mask]
# validPtsnew['KExcit'] = validPts['KExcit'].to_numpy()[Mask]
# validPtsnew['Idx_i'] = validPts['Idx_i'].to_numpy()[Mask]
# validPtsnew['Idx_j'] = validPts['Idx_j'].to_numpy()[Mask]
# validPtsnew['yWeights'] = validPts['yWeights'].to_numpy()[Mask]
# validPtsnew['qi'] = validPts['qi'].to_numpy()[Mask]
# validPtsnew['qj'] = validPts['qj'].to_numpy()[Mask]
# validPtsnew['partition_function_ratio'] = validPts['partition_function_ratio'].to_numpy()[Mask]
# validPtsnew.to_csv(PathToValidPts + '/res.csv', index=False)


trainPtsnew = trainPts.copy()
trainPtsnew['qi']  = DiatData[0]['g'].to_numpy()[trainPts['Idx_i']] * np.exp( - DiatData[0]['EInt'].to_numpy()[trainPts['Idx_i']] / (kB*trainPts['TTran_i']) )
trainPtsnew['qj']  = DiatData[0]['g'].to_numpy()[trainPts['Idx_j']] * np.exp( - DiatData[0]['EInt'].to_numpy()[trainPts['Idx_j']] / (kB*trainPts['TTran_i']) )
trainPtsnew['partition_function_ratio'] = trainPtsnew['qj']/trainPtsnew['qi']
trainPtsnew.to_csv(PathToTrainPts + '/res.csv', index=False)

validPtsnew = validPts.copy()
validPtsnew['qi']  = DiatData[0]['g'].to_numpy()[validPts['Idx_i']] * np.exp( - DiatData[0]['EInt'].to_numpy()[validPts['Idx_i']] / (kB*validPts['TTran_i']) )
validPtsnew['qj']  = DiatData[0]['g'].to_numpy()[validPts['Idx_j']] * np.exp( - DiatData[0]['EInt'].to_numpy()[validPts['Idx_j']] / (kB*validPts['TTran_i']) )
validPtsnew['partition_function_ratio'] = validPtsnew['qj']/validPtsnew['qi']
validPtsnew.to_csv(PathToValidPts + '/res.csv', index=False)


In [10]:
np.amin(trainPtsnew['partition_function_ratio'].to_numpy())

0.0010001169303306

In [14]:
trainPts['TTran_i']

0          5000.0
1         20000.0
2          5000.0
3         10000.0
4         20000.0
           ...   
617443     7500.0
617444    12000.0
617445     7500.0
617446     7500.0
617447    15000.0
Name: TTran_i, Length: 617448, dtype: float64